In [ ]:
import socket

def receive_nmea_msg_gps(HOST='192.168.3.1', PORT=28784, time='sec1'):
    """_summary_

    Args:
        HOST (str, optional): IP address of receiver. Defaults to '192.168.3.1'.
        PORT (int, optional): TCP/IP PORT. Defaults to 28784.
        time (str, optional): regularity of NMEA messages. Defaults to 'sec1'.
    """''

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        
        # Connect to the GPS 
        s.connect((HOST, PORT))
        
        # Receive the command line: if all good
        data = s.recv(64)

        # Establish a stream for this established connection
        command_msg = 'sno, Stream 3, IP11, GGA, ' + time + '\n'
        s.sendall(command_msg.encode('utf-8'))
        
        
        for i in range(5):
            data = s.recv(64)
            print(data)


In [50]:
import socket

HOST = "192.168.3.1"  # The server's hostname or IP address
PORT = 28784 # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    #s.sendall(b"Hello, world")
    data = s.recv(64)

    print(data)

    s.sendall('sno, Stream 3, IP11, GGA, sec1\n'.encode('utf-8'))
    
    for i in range(5):
        data = s.recv(64)
        print(data)

print(7)

OSError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions

In [2]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

In [3]:


def ask_for_port():
    """\
    Show a list of ports and ask the user for a choice. To make selection
    easier on systems with long device names, also allow the input of an
    index.
    """
    sys.stderr.write('\n--- Available ports:\n')
    ports = []
    for n, (port, desc, hwid) in enumerate(sorted(comports()), 1):
        sys.stderr.write('--- {:2}: {:20} {!r}\n'.format(n, port, desc))
        ports.append(port)
    while True:
        sys.stderr.write('--- Enter port index or full name: ')
        port = input('')
        try:
            index = int(port) - 1
            if not 0 <= index < len(ports):
                sys.stderr.write('--- Invalid index!\n')
                continue
        except ValueError:
            pass
        else:
            port = ports[index]
        return port
 
 

In [34]:
port = ask_for_port()
print(port, type(port))


--- Available ports:
---  1: COM3                 'Intel(R) Active Management Technology - SOL (COM3)'
---  2: COM11                'Septentrio Virtual USB COM Port 1 (COM11)'
---  3: COM12                'Septentrio Virtual USB COM Port 2 (COM12)'
--- Enter port index or full name: 

COM11 <class 'str'>


In [35]:
serial_instance = None
while serial_instance is None:
    try:
        serial_instance = serial.serial_for_url(port,
                                                9600,
                                                parity='N',
                                                rtscts=False,
                                                xonxoff=False,
                                                do_not_open=True)

        
        serial_instance.timeout = 1
                    
        serial_instance.exclusive = True
        serial_instance.open()
        
        print('we did it')
        
        
    except serial.SerialException as e:
        sys.stderr.write('could not open port {!r}: {}\n'.format(port, e))

    else:
        print('We enter')
        break

print('loop is over')
        

we did it
We enter
loop is over


In [36]:
import threading
import pynmea2

buffer = []
cnt = 0

def process_data(data):
    try:
        nmeaobj = pynmea2.parse(data.decode('utf-8'))
        extracted_data = ['%s: %s' % (nmeaobj.fields[i][0], nmeaobj.data[i]) for i in range(len(nmeaobj.fields))]
    except:
        # Do not save any other comand line
        return
    
    #extracted_data.append('Num_Line_RX: ' + str(cnt_here))
    buffer.append(extracted_data)
    
def receive_serial(serial_instance_actual, stop_event):
    while not stop_event.is_set():
        rx_msg = serial_instance_actual.readline()
        #cnt = cnt+1
        if rx_msg is not None:
            process_data(rx_msg)

In [37]:
event_stop_thread_serial = threading.Event()
t_receive = threading.Thread(target=receive_serial, args=(serial_instance, event_stop_thread_serial))
t_receive.start()

In [43]:
event_stop_thread_serial.set()

In [38]:
serial_instance.write('sno, Stream 3, USB1, GGA, sec1\n'.encode('utf-8'))

31

In [46]:
serial_instance.close()

In [ ]:
import pynmea2

nmeaobj = pynmea2.parse(data.decode('utf-8'))
['%s: %s' % (nmeaobj.fields[i][0], nmeaobj.data[i]) 
     for i in range(len(nmeaobj.fields))]

In [1]:
from a2gmeasurements import GpsSignaling

mySeptentrio = GpsSignaling(1)
mySeptentrio.serial_connect()

mySeptentrio.start_thread_serial()

mySeptentrio.sendCommandGps('sno, Stream 3, USB1, GGA, sec1')


In [6]:
mySeptentrio.stop_thread_serial()

In [7]:
len(mySeptentrio.gps_rx_buffer)

18

In [9]:
mySeptentrio.gps_rx_buffer[0]['Timestamp']

''

In [1]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.sepectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()

print(rx_power)

mySA.spectrum_analyzer_close()



{'MAG': '-50.03\n', 'MAG_UNITS': 'dBm', 'FREQ': '60032000000', 'FREQ_UNITS': 'Hz'}
